\begin{equation}
P(x)
\end{equation}

| | Movie 1| Movie 2 | Movie 3 | Movie 4 | Movie 5|
|--|--|--|--|--|--|
| User 1| 4 | 4 | 0 | 2 | 2 |
| User 2| 4 | 0 | 0 | 3 | 3 |
| User 3| 4 | 0 | 0 | 1 | 1 |
| User 4| 1 | 1 | 1 | 2 | 0 |
| User 5| 2 | 2 | 2 | 0 | 0 |
| User 6| 1 | 1 | 1 | 0 | 0 |
| User 7| 5 | 5 | 5 | 0 | 0 |



\begin{equation}
\begin{aligned}
s_1 = & CosSim(\begin{bmatrix}
4 \\ 
1 \\ 
2 \\ 
1 \\
5
\end{bmatrix},\begin{bmatrix}
4 \\ 
1 \\ 
2 \\ 
1 \\
5
\end{bmatrix})=1 \\
s_4 = & CosSim(\begin{bmatrix}
2 \\ 
2
\end{bmatrix},\begin{bmatrix}
4 \\ 
1
\end{bmatrix})=0.93 \\
s_5 = & CosSim(2,4)=1
\end{aligned}
\end{equation}



$$
Score(\text{Movie 2})=\frac{4*1+1*0.93+1*1}{1+0.93+1}=2.02
$$

同理可计算Moview 3的评分为：
$$
Score(\text{Movie 3})=2.50
$$

所以给用户3推荐的电影顺序为：Movie 3、Movie 2。
